## Q.1. 通道交换

读取图像，然后将 RGB 通道替换成 BGR 通道。

下面的代码用于提取图像的红色通道。注意，`cv2.imread()` 的系数是按 BGR 顺序排列的！其中的变量 `red` 表示的是仅有原图像红通道的 `imori.jpg`。

In [3]:
import cv2
img = cv2.imread("imori.jpg")
red = img[:,:,2].copy()
blue = img[:,:,0].copy()
green = img[:,:,1].copy()
img[:,:,0]=red
img[:,:,1]=green
img[:,:,2]=blue
cv2.imshow('',img)
cv2.waitKey(0)

-1

## Q.2. 灰度化（Grayscale）

将图像灰度化吧！灰度是一种图像亮度的表示方法，通过下式计算：

Y = 0.2126 R + 0.7152 G + 0.0722 B

In [14]:
import cv2
import numpy as np
img = cv2.imread("imori.jpg")
red = img[:,:,2].copy()
blue = img[:,:,0].copy()
green = img[:,:,1].copy()
Y = 0.2126*red+0.7152*green+0.0722*blue #dtype('float64')
Y = Y.astype(np.uint8)
cv2.imshow('',Y)
cv2.waitKey(0)

-1

## Q.3. 二值化（Thresholding）

把图像进行二值化吧！二值化是将图像使用黑和白两种值表示的方法。这里我们将灰度的阈值设置为 128 来进行二值化，即：

```bash
y = { 0 (if y < 128)
     255 (else) 
```

In [18]:
import cv2
import numpy as np
img = cv2.imread("imori.jpg")
red = img[:,:,2].copy()
blue = img[:,:,0].copy()
green = img[:,:,1].copy()
Y = 0.2126*red+0.7152*green+0.0722*blue #dtype('float64')
Y = Y.astype(np.uint8)
H,W = Y.shape
y=128
"""
for i in range(0,H):
    for j in range(0,W):
        if Y[i,j]<y:
            Y[i,j]=0
        else:
            Y[i,j]=255
"""
Y[Y<y]=0
Y[Y>y]=255
cv2.imshow('',Y)
cv2.waitKey(0)

-1

## Q.4. 大津二值化算法（Otsu's Method）

使用大津算法来二值化图像吧！大津算法，也被称作最大类间方差法，是一种可以自动确定二值化中阈值的算法，从类内方差和类间方差的比值计算得来：


- 小于阈值 t 的类记作 0，大于阈值 t 的类记作 1；
- w0 和 w1 是被阈值 t 分开的两个类中的像素数占总像素数的比率（满足 w0+w1=1）；
- S0^2, S1^2 是这两个类中像素值的方差；
- M0, M1 是这两个类的像素值的平均值；

也就是说：

```bash
类内方差：Sw^2 = w0 * S0^2 + w1 * S1^2
类间方差：Sb^2 = w0 * (M0 - Mt)^2 + w1 * (M1 - Mt)^2 = w0 * w1 * (M0 - M1) ^2
图像所有像素的方差：St^2 = Sw^2 + Sb^2 = (const)
根据以上的式子，我们用以下的式子计算分离度：  
分离度 X = Sb^2 / Sw^2 = Sb^2 / (St^2 - Sb^2)
```

也就是说： 

```bash
argmax_{t} X = argmax_{t} Sb^2
```
换言之，如果使``` Sb^2 =  w0 * w1 * (M0 - M1) ^2 ```最大，就可以得到最好的二值化阈值 t。



它是按图像的灰度特性,将图像分成背景和目标两部分，或者说，是寻找一个阈值为K，将图像的颜色分为1,2.....K和K+1.....256两部分。

如何确定这个阈值K？算法分类的原理是让背景和目标之间的类间方差最大，因为背景和目标之间的类间方差越大,说明构成图像的2部分的差别越大,错分的可能性越小。下面进行公式推导：

首先是符号说明：对于图像I(x,y),前景(即目标)和背景的分割阈值记作T,属于前景的像素点数占整幅图像的比例记为ω1,其平均灰度μ1;背景像素点数占整幅图像的比例为ω2,其平均灰度为μ2。图像的总平均灰度记为μ,类间方差记为g。假设图像的背景较暗,并且图像的大小为M×N,图像中像素的灰度值小于阈值T的像素个数记作N1,像素灰度大于阈值T的像素个数记作N2。

```bash
ω1 = N1/M×N
ω2 = N2/M×N
N1+N2 = M×N
ω1+ω2 = 1
μ = μ1xω1 + μ2xω2
g = ω1x(μ-μ1)^2 + ω2x(μ-μ2)^2
=>g = ω1xω2x(μ1-μ2)^2
```

In [24]:
import cv2
import numpy as np
img = cv2.imread("imori.jpg")
red = img[:,:,2].copy()
blue = img[:,:,0].copy()
green = img[:,:,1].copy()
Y = 0.2126*red+0.7152*green+0.0722*blue #dtype('float64')
Y = Y.astype(np.uint8)
H,W=Y.shape
max_sigma = 0
max_t = 0

for _t in range(1, 255):
    v0 = Y[np.where(Y < _t)]
    if len(v0) != 0:
        m0 = np.mean(v0) 
    w0 = len(v0) / (H * W)
    v1 = Y[np.where(Y >= _t)]
    m1 = np.mean(v1) if len(v1) > 0 else 0.
    w1 = len(v1) / (H * W)
    sigma = w0 * w1 * ((m0 - m1) ** 2)
    if sigma > max_sigma:
        max_sigma = sigma
        max_t = _t
print("threshold >>", max_t)
th = max_t
Y[Y < th] = 0
Y[Y >= th] = 255
cv2.imshow('',Y)
cv2.waitKey(0)

threshold >> 127


-1

## Q.5. HSV 变换

将使用 HSV 表示色彩的图像的色相反转吧！

HSV 即使用**色相（Hue）、饱和度（Saturation）、明度（Value）**来表示色彩的一种方式。

- 色相：将颜色使用0到360度表示，就是平常所说的颜色名称，如红色、蓝色。色相与数值按下表对应：

  | 红  | 黄  | 绿  | 青色 | 蓝色 | 品红 | 红  |  
  | 0   | 60  | 120 | 180  | 240  | 300  | 360 |  

- 饱和度：是指色彩的纯度，饱和度越低则颜色越黯淡( 0<= S < 1)；
- 明度：即颜色的明暗程度。数值越高越接近白色，数值越低越接近黑色 ( 0 <= V < 1)；

从 RGB 色彩表示转换到 HSV 色彩表示通过以下方式计算：

R,G,B的值在[0, 1]之间：

```bash
Max = max(R,G,B)
Min = min(R,G,B)

H =  { 0                            (if Min=Max)
       60 x (G-R) / (Max-Min) + 60  (if Min=B)
       60 x (B-G) / (Max-Min) + 180 (if Min=R)
       60 x (R-B) / (Max-Min) + 300 (if Min=G)
       
V = Max

S = Max - Min
```

从 HSV 色彩表示转换到 RGB 色彩表示通过以下方式计算：

```bash
C = S

H' = H / 60

X = C (1 - |H' mod 2 - 1|)

(R,G,B) = (V - C) (1,1,1) + { (0, 0, 0)  (if H is undefined)
                              (C, X, 0)  (if 0 <= H' < 1)
                              (X, C, 0)  (if 1 <= H' < 2)
                              (0, C, X)  (if 2 <= H' < 3)
                              (0, X, C)  (if 3 <= H' < 4)
                              (X, 0, C)  (if 4 <= H' < 5)
                              (C, 0, X)  (if 5 <= H' < 6)
```
请将色相反转（色相值加180），然后再用 RGB 色彩空间表示图片。

In [73]:

import cv2
import numpy as np
img = cv2.imread("imori.jpg")/255.0
red = img[:,:,2].copy()
blue = img[:,:,0].copy()
green = img[:,:,1].copy()
out = np.zeros_like(img)
max_v = np.maximum(red,np.maximum(blue,green))
#max_v = np.max(img, axis=2).copy()
min_v = np.minimum(red,np.minimum(blue,green))
#min_v = np.min(img,axis = 2).copy()
min_arg = np.argmin(img, axis=2)
H = np.zeros_like(max_v)
H[np.where(max_v == min_v)]=0
## if min == B
ind = np.where(min_arg == 0)
H[ind] = 60 * (green[ind] - red[ind]) / (max_v[ind] - min_v[ind]) + 60
## if min == R
ind = np.where(min_arg == 2)
H[ind] = 60 * (blue[ind] - green[ind]) / (max_v[ind] - min_v[ind]) + 180
## if min == G
ind = np.where(min_arg == 1)
H[ind] = 60 * (red[ind] - blue[ind]) / (max_v[ind] - min_v[ind]) + 300


V = max_v.copy()
S =max_v.copy()-min_v.copy()

H = (H+180)%360

C = S

H1 = H / 60

X = C*(1 - np.abs( H1 % 2 - 1))

Z = np.zeros_like(H)

val = [[C,X,Z],[X,C,Z],[Z,C,X],[Z,X,C],[X,Z,C],[C,Z,X]]
#val = [[Z,X,C], [Z,C,X], [X,C,Z], [C,X,Z], [C,Z,X], [X,Z,C]]
for i in range(6):
    ind = np.where((i <= H1) & (H1 < (i+1)))
    out[:,:,0][ind] = (V -C)[ind] +val[i][0][ind]
    out[:,:,1][ind] = (V-C)[ind] +val[i][1][ind]
    out[:,:,2][ind] = (V-C)[ind]+val[i][2][ind]
out[np.where(max_v ==min_v)]=0
out = (out * 255).astype(np.uint8) 
cv2.imshow("result", out)
cv2.waitKey(0)



D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in remainder
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in remainder
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in greater_equal
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less


-1

## Q.6. 减色处理

> 这里没有找到"減色処理"准确的中文翻译，所以直译了。
>
> ——gzr

这里我们将图像的值由256^3压缩至4^3，即将 RGB 的值只取 {32, 96, 160, 224}。这被称作色彩量化。色彩的值按照下面的方式定义：

```bash
val = {  32  (  0 <= val <  64)
         96  ( 64 <= val < 128)
        160  (128 <= val < 192)
        224  (192 <= val < 256)
```

In [95]:
import cv2

img = cv2.imread("imori.jpg")
out = img.copy()
out1 = img.copy()

for i in range(len(out)):
    for j in range(len(out)):
        for k in range(3):
            if ((0 <= out[i,j,k]) & (out[i,j,k] < 64)):
                out[i,j,k] = 32
            elif ((64 <= out[i,j,k]) & (out[i,j,k] < 128)):
                out[i,j,k] = 96
            elif ((128 <= out[i,j,k]) & (out[i,j,k] < 192)):
                out[i,j,k] = 160
            else:
                out[i,j,k] = 224

out1 = out1 // 64 * 64 + 32
cv2.imshow("",out1)
cv2.waitKey(0)

-1

## Q.7. 平均池化（Average Pooling）

将图片按照固定大小网格分割，网格内的像素值取网格内所有像素的平均值。我们将这种把图片使用均等大小网格分割，并求网格内代表值的操作称为池化（Pooling）。池化操作是卷积神经网络（Convolutional Neural Network）中重要的图像处理方式。平均池化按照下式定义：

```bash
v = 1/|R| * Sum_{i in R} v_i
```

请把大小为 128x128 的`imori.jpg`使用 8x8 的网格做平均池化。

In [97]:
import cv2
import numpy as np
img = cv2.imread('imori.jpg')
out = img.copy()

H,W,C = out.shape
G = 8
Nh = int(H/G)
Nw = int(W/G)

for i in range(Nh):
    for j in range(Nw):
        for k in range(C):
            out[G*i:G*(i+1), G*j:G*(j+1), k] = np.mean(out[G*i:G*(i+1), G*j:G*(j+1), k]).astype(np.int)
cv2.imshow('',out)
cv2.waitKey(0)

-1

## Q.8. 最大池化（Max Pooling）

网格内的值不取平均值，而是取网格内的最大值进行池化操作。

In [98]:
import cv2
import numpy as np
img = cv2.imread('imori.jpg')
out = img.copy()

H,W,C = out.shape
G = 8
Nh = int(H/G)
Nw = int(W/G)

for i in range(Nh):
    for j in range(Nw):
        for k in range(C):
            out[G*i:G*(i+1), G*j:G*(j+1), k] = np.max(out[G*i:G*(i+1), G*j:G*(j+1), k]).astype(np.int)
            
cv2.imshow('',out)
cv2.waitKey(0)

-1

## Q.9. 高斯滤波（Gaussian Filter）

使用高斯滤波器（3x3 大小，标准差 s=1.3​）来对`imori_noise.jpg`进行降噪处理吧！

高斯滤波器是一种可以使图像平滑的滤波器，用于去除噪声。可用于去除噪声的滤波器还有中值滤波器（参见问题10），平滑滤波器（参见问题11）、LoG 滤波器（参见问题19）。

高斯滤波器将中心像素周围的像素按照高斯分布加权平均进行平滑化。这样的（二维）权值通常被称为卷积核或者滤波器。

但是，由于图像的长宽可能不是滤波器大小的整数倍，因此我们需要在图像的边缘补0。这种方法称作 Zero Padding。并且权值（卷积核）要进行[归一化操作](https://blog.csdn.net/lz0499/article/details/54015150)(sum g = 1)。

```bash
权值 g(x,y,s) = 1/ (s*sqrt(2 * pi)) * exp( - (x^2 + y^2) / (2*s^2))
标准差 s = 1.3 的 8 近邻 高斯滤波器如下：
            1 2 1
K =  1/16 [ 2 4 2 ]
            1 2 1
```

In [107]:
import cv2

img = cv2.imread('imori_noise.jpg')
H,W,C = img.shape

K_size = 3
s = 1.3

pad = K_size // 2
out = np.zeros((H + pad*2, W + pad*2, C), dtype=np.float)
out[pad:pad+H,pad:pad+W] = img.copy().astype(np.float)

K = np.zeros((K_size, K_size), dtype=np.float)
for i in range(-pad, -pad+K_size):
    for j in range(-pad, -pad+K_size):
        K[j+pad, i+pad] = np.exp( -(i**2 + j**2) / (2* (sigma**2)))
#print(K)        
K /= (sigma * np.sqrt(2 * np.pi))
#print(K) 
K /= K.sum()
tmp = out.copy()

for i in range(H):
    for j in range(W):
        for k in range(C):
            out[pad+i, pad+j, k] = np.sum(K * tmp[i:i+K_size, j:j+K_size, k])

out = out[pad:pad+H, pad:pad+W].astype(np.uint8)
cv2.imshow('',out)
cv2.waitKey(0)

-1

## Q.10 中值滤波（Median filter）

使用中值滤波器（3x 3大小）来对`imori_noise.jpg`进行降噪处理吧！

中值滤波器是一种可以使图像平滑的滤波器。这种滤波器用滤波器范围内（在这里是3x3）像素点的中值进行滤波，在这里也采用 Zero Padding。

In [108]:
import cv2
import numpy as np

# Read image
img = cv2.imread("imori_noise.jpg")
H, W, C = img.shape


# Median Filter
K_size = 3

## Zero padding
pad = K_size // 2
out = np.zeros((H + pad*2, W + pad*2, C), dtype=np.float)
out[pad:pad+H, pad:pad+W] = img.copy().astype(np.float)

tmp = out.copy()

for i in range(H):
    for j in range(W):
        for k in range(C):
            out[pad+i, pad+j, k] = np.median(tmp[i:i+K_size, j:j+K_size, k])

out = out[pad:pad+H, pad:pad+W].astype(np.uint8)

# Save result

cv2.imshow("result", out)
cv2.waitKey(0)
cv2.destroyAllWindows()
